In [1]:
import obi_one as obi
import os
import pandas as pd

In [2]:
input_root = "/Users/pokorny/Data/Circuits"
output_root = "../../../obi-output/extracted_single_neuron_circuits"

In [3]:
# Number of single-neuron circuits to extract
idx_list = [0, 1]
node_selection_file = "./top_nodes_dim6.csv"
node_id_lists = [obi.NamedTuple(name=f"SingleNeuronCircuit__" + os.path.splitext(os.path.split(node_selection_file)[1])[0] + f"__IDX{_idx}", elements=tuple([pd.read_csv(node_selection_file)["node_ids"].to_list()[_idx]])) for _idx in idx_list]

In [ ]:
circuit_extractions_form = obi.CircuitExtractions(
                    initialize=obi.CircuitExtractions.Initialize(
                        # circuit=obi.Circuit(name="ToyCircuit-S1-6k", path=os.path.join(input_root, "ToyCircuit-S1-6k", "circuit_config.json")),
                        circuit=obi.Circuit(name="nbS1-HEX0", path=os.path.join(input_root, "nbS1-HEX0", "circuit_config.json")),
                        run_validation=False,
                    ),
                    # neuron_set=obi.PredefinedNeuronSet(node_set=["Excitatory", "Inhibitory"], sample_percentage=[50, 100]),
                    # neuron_set=obi.PredefinedNeuronSet(node_set=["Layer2", "Layer3", "Layer4", "Layer5", "Layer6"], sample_percentage=10),
                    neuron_set=obi.IDNeuronSet(neuron_ids=node_id_lists)
)

grid_scan = obi.GridScan(form=circuit_extractions_form, output_root=output_root, coordinate_directory_option="VALUE")

In [5]:
# Display the intermediary data
grid_scan.multiple_value_parameters(display=True)
grid_scan.coordinate_parameters(display=True)
grid_scan.coordinate_instances(display=True)


MULTIPLE VALUE PARAMETERS
neuron_set.neuron_ids: [SingleNeuronCircuit__top_nodes_dim6__IDX0, SingleNeuronCircuit__top_nodes_dim6__IDX1]

COORDINATE PARAMETERS
neuron_set.neuron_ids: SingleNeuronCircuit__top_nodes_dim6__IDX0
neuron_set.neuron_ids: SingleNeuronCircuit__top_nodes_dim6__IDX1

COORDINATE INSTANCES
CircuitExtraction(idx=0, scan_output_root=PosixPath('.'), coordinate_output_root=PosixPath('.'), single_coordinate_scan_params=SingleCoordinateScanParams(type='SingleCoordinateScanParams', scan_params=[SingleValueScanParam(type='SingleValueScanParam', location_list=['neuron_set', 'neuron_ids'], value=SingleNeuronCircuit__top_nodes_dim6__IDX0)], nested_coordinate_subpath_str=PosixPath('.')), type='CircuitExtraction', neuron_set=IDNeuronSet(type='IDNeuronSet', simulation_level_name=None, sample_percentage=None, sample_seed=0, neuron_ids=SingleNeuronCircuit__top_nodes_dim6__IDX0), initialize=Initialize(type='CircuitExtractions.Initialize', circuit=Circuit(type='Circuit', name='nbS1-

[CircuitExtraction(idx=0, scan_output_root=PosixPath('.'), coordinate_output_root=PosixPath('.'), single_coordinate_scan_params=SingleCoordinateScanParams(type='SingleCoordinateScanParams', scan_params=[SingleValueScanParam(type='SingleValueScanParam', location_list=['neuron_set', 'neuron_ids'], value=SingleNeuronCircuit__top_nodes_dim6__IDX0)], nested_coordinate_subpath_str=PosixPath('.')), type='CircuitExtraction', neuron_set=IDNeuronSet(type='IDNeuronSet', simulation_level_name=None, sample_percentage=None, sample_seed=0, neuron_ids=SingleNeuronCircuit__top_nodes_dim6__IDX0), initialize=Initialize(type='CircuitExtractions.Initialize', circuit=Circuit(type='Circuit', name='nbS1-HEX0', path='/Users/pokorny/Data/Circuits/nbS1-HEX0/circuit_config.json'), run_validation=False)),
 CircuitExtraction(idx=1, scan_output_root=PosixPath('.'), coordinate_output_root=PosixPath('.'), single_coordinate_scan_params=SingleCoordinateScanParams(type='SingleCoordinateScanParams', scan_params=[SingleVal

In [6]:
grid_scan.execute(processing_method='run')

Extracting subcircuit from 'nbS1-HEX0'
Copying morphologies for population 'S1nonbarrel_neurons' (1)


Copying containerized .h5 morphologies: 100%|████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 1082.12it/s]

Copied 1 morphologies into container (0 already existed)
Copying biophysical_neuron_models for population 'S1nonbarrel_neurons' (1)
Copying mod files
Extraction DONE
Extracting subcircuit from 'nbS1-HEX0'


Copying morphologies for population 'S1nonbarrel_neurons' (1)


Copying containerized .h5 morphologies: 100%|████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 1558.06it/s]

Copied 1 morphologies into container (0 already existed)
Copying biophysical_neuron_models for population 'S1nonbarrel_neurons' (1)
Copying mod files
Extraction DONE
create_bbp_workflow_campaign_config() not yet complete.


{0: None, 1: None}

In [7]:
# Check circuits
from bluepysnap import Circuit

for _inst in grid_scan.coordinate_instances():
    cfg = grid_scan.output_root / str(_inst.neuron_set.neuron_ids) / "circuit_config.json"
    c = Circuit(cfg)
    local_nsyn = vpm_nsyn = pom_nsyn = 0
    if "S1nonbarrel_neurons__S1nonbarrel_neurons__chemical" in c.edges.population_names:
        local_nsyn = c.edges["S1nonbarrel_neurons__S1nonbarrel_neurons__chemical"].size
    if "VPM__S1nonbarrel_neurons__chemical" in c.edges.population_names:
        vpm_nsyn = c.edges["VPM__S1nonbarrel_neurons__chemical"].size
    if "POm__S1nonbarrel_neurons__chemical" in c.edges.population_names:
        pom_nsyn = c.edges["POm__S1nonbarrel_neurons__chemical"].size
    print(f"{_inst.neuron_set.neuron_ids}: {c.nodes["S1nonbarrel_neurons"].size} neuron, {local_nsyn} local synapses, {vpm_nsyn} VPM synapses, {pom_nsyn} POm synapses")


SingleNeuronCircuit__top_nodes_dim6__IDX0: 1 neuron, 0 local synapses, 0 VPM synapses, 0 POm synapses
SingleNeuronCircuit__top_nodes_dim6__IDX1: 1 neuron, 0 local synapses, 104 VPM synapses, 88 POm synapses
